In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.special import expit as sigmoid  # logistic function
import variablen as var
# The following code is work in progress
from sklearn.preprocessing import normalize

In [2]:
X_complete = pd.read_csv(var.prepaired_df, delimiter=";", decimal=",")

In [3]:
# Quelle https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values

def ampute_mar(X_complete, missing_rate, W=None):
    """ Observed values will censor the missing ones
    
    The proba of being missing: M_proba = X_obs.dot(W)
    So for each sample, some observed feature (P=1) will influence 
    the missingness of some others features (P=0) w.r.t to the weight 
    matrix W (shape n_features x n_features).
    
    e.g. during a questionnary, those who said being busy (X_obs[:,0] = 1) 
    usualy miss to fill the last question (X_obs[:,-1] = np.nan)
    So here W[0,-1] = 1
    """
    X_obs = X_complete.copy()
    M_proba = np.zeros(X_obs.shape)
    
    if W is None:
        # generate the weigth matrix W
        W = np.random.randn(X_complete.shape[1], X_complete.shape[1])

    # Severals iteration to have room for high missing_rate
    for i in range(X_obs.shape[1]*2):
        # Sample a pattern matrix P
        # P[i,j] = 1 will correspond to an observed value
        # P[i,j] = 0 will correspond to a potential missing value
        P = np.random.binomial(1, .5, size=X_complete.shape)

        # potential missing entry do not take part of missingness computation
        X_not_missing = np.multiply(X_complete,P)

        # sample from the proba X_obs.dot(W)
        sigma = np.var(X_not_missing)
        M_proba_ = np.random.normal(X_not_missing.dot(W), scale = sigma)

        # not missing should have M_proba = 0
        M_proba_ = np.multiply(M_proba_, 1-P)  # M_proba[P] = 0
        
        M_proba += M_proba_

    thresold = np.percentile(M_proba.ravel(), 100 * (1 - missing_rate))
    M = M_proba > thresold

    np.putmask(X_obs, M, np.nan)
    print('Percentage of newly generated mising values: {}'.\
      format(np.sum(np.isnan(X_obs))/X_obs.size))
    return X_obs

In [4]:
W = np.array([[0,10],[0,0]]) 
# With this weight matrix W, 
# Numpy seed random
np.random.seed(var.random_seed)
# missingness of X[:,1] depends on X[:,0] values
for missing_rate in var.missing_rats:
    X_obs_mar = pd.DataFrame(ampute_mar(X_complete.to_numpy() ,missing_rate))
    X_obs_mar.columns = [X_complete.columns]
    filename = str(np.int_(100 *  missing_rate))
    X_obs_mar.to_csv(var.mar_dir + filename + ".csv" ,index=False, sep=';', decimal=',')

Percentage of newly generated mising values: 0.1000000134129724
Percentage of newly generated mising values: 0.2000000268259448
Percentage of newly generated mising values: 0.30000004023891724
Percentage of newly generated mising values: 0.4000000536518896
Percentage of newly generated mising values: 0.49999993293513795
Percentage of newly generated mising values: 0.5999999463481104
Percentage of newly generated mising values: 0.6999999597610828
Percentage of newly generated mising values: 0.7999999731740551
